# CS-GY 6953 Deep Learning Mini-Project Shallow network

Task: Building a ResNet model on CIFAR-10 with a constraint of 5 million parameters maximum.

## 1. Starting Simple: Load the CIFAR-10 dataset and build the first shallow ResNet network

Create the data loaders.

In [1]:
!pip install torchvision
!pip install torchsummary

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

In [3]:
batch_size = 128

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])
# There are more functions for augmentation.
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(root='./data',train=False, download=True, transform=transform_test)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


Construct the ResNet.

In [4]:
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchsummary import summary

import os

In [5]:
class BasicBlock(nn.Module): # A residual block with 2 convolutional layers and a skip connection.
    expansion = 1
    
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes,      # Number of channels in the input image
                               planes,         # Number of output channels
                               kernel_size=3,  # Size of the convolution filter
                               stride=stride,  # Stride of the convolution
                               padding=1,      # Padding added to all four sides of the input, int or {'valid', 'same'}, etc.
                               # padding_mode={'zeros', 'reflect', 'replicate', 'circular'}
                               bias=False)     # If true, adds a learnable bias ot the output.
        self.bn1 = nn.BatchNorm2d(planes)      # Other parameters include eps, momentum, affine, etc.
                                               # affine=True: affine parameters being learnable.
        self.conv2 = nn.Conv2d(planes, 
                               planes, 
                               kernel_size=3,
                               stride=1,
                               padding=1,
                               bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.shortcut = nn.Sequential()        # A sequentil wrapper of layers.
        # If the dimentions of skip connection input and output don't match, do the following.
        # As an alternative solution, zero-padding is also proposed in the paper.
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(nn.Conv2d(in_planes, 
                                                    self.expansion*planes, 
                                                    kernel_size=1, 
                                                    stride=stride, 
                                                    bias=False),
                                          nn.BatchNorm2d(self.expansion*planes))
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [6]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64
        
        self.conv1 = nn.Conv2d(3, 64, 
                               kernel_size=3, 
                               stride=1, 
                               padding=1, 
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 96, num_blocks[3], stride=2)
        self.layer5 = self._make_layer(block, 128, num_blocks[4], stride=2)
        self.layer6 = self._make_layer(block, 160, num_blocks[5], stride=2)
        self.layer7 = self._make_layer(block, 192, num_blocks[6], stride=2)
        self.layer8 = self._make_layer(block, 220, num_blocks[6], stride=2)
        # self.linear = nn.Linear(512*block.expansion, num_classes)
        self.linear = nn.Linear(220*block.expansion, num_classes)
        
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes*block.expansion
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        # out = F.avg_pool2d(out, 4)
        out = F.avg_pool2d(out, 1)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [7]:
 model = ResNet(BasicBlock, [2,2,2,2,2,2,2])

In [8]:
summary(model, (3, 32, 32), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 16, 32, 32]           9,216
       BatchNorm2d-4           [-1, 16, 32, 32]              32
            Conv2d-5           [-1, 16, 32, 32]           2,304
       BatchNorm2d-6           [-1, 16, 32, 32]              32
            Conv2d-7           [-1, 16, 32, 32]           1,024
       BatchNorm2d-8           [-1, 16, 32, 32]              32
        BasicBlock-9           [-1, 16, 32, 32]               0
           Conv2d-10           [-1, 16, 32, 32]           2,304
      BatchNorm2d-11           [-1, 16, 32, 32]              32
           Conv2d-12           [-1, 16, 32, 32]           2,304
      BatchNorm2d-13           [-1, 16, 32, 32]              32
       BasicBlock-14           [-1, 16,

Note: No non-linearity in the fully connected part.

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"The device is {device}")
model = model.to(device)
if device == 'cuda':
    model = torch.nn.DataParallel(model)
    cudnn.benchmark = True

The device is cuda


In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), 
                      lr=0.1, 
                      momentum=0.9, 
                      weight_decay=5e-4)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", factor=0.1, patience=15)

In [11]:
model_chk_name =  'deep_8layer_ckpt'

In [12]:
def train(epoch):
    print('\nEpoch: %d'%epoch)
    model.train()
    train_loss = 0
    correct, total = 0, 0
    for (inputs, targets) in trainloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    acc = correct/total*100
    train_loss /= total
    print(f"Train | Loss: {round(train_loss, 4)} | Acc: {round(acc, 2)}")
    train_losses.append(train_loss)
    train_accs.append(acc)

def test(epoch):
    global train_losses, test_losses, train_accs, test_accs, lrs, best_acc
    model.eval()
    test_loss = 0
    correct, total = 0, 0
    with torch.no_grad():
        for (inputs, targets) in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    acc = correct/total*100
    test_loss /= total
    print(f"Test  | Loss: {round(test_loss, 4)} | Acc: {round(acc, 2)}")
    test_losses.append(test_loss)
    test_accs.append(acc)
    
    state = {'model': model.state_dict(),
             'train_loss': train_losses,
             'test_loss': test_losses,
             'train_acc': train_accs,
             'test_acc': test_accs,
             'lrs':lrs,
             'epoch': epoch}
    torch.save(state, './checkpoint/'+model_chk_name+'.pth')
    if acc > best_acc:
        torch.save(state, './checkpoint/'+model_chk_name+'_best.pth')
        best_acc = acc

if not os.path.isdir('checkpoint'):
    os.mkdir('checkpoint')
state = {'model': model.state_dict(),
         'train_loss': [],
         'test_loss': [],
         'train_acc': [],
         'test_acc': [],
         'lr':[],
         'epoch': -1}
torch.save(state, './checkpoint/'+model_chk_name+'.pth')
torch.save(state, './checkpoint/'+model_chk_name+'_best.pth')

In [13]:
checkpoint = torch.load('./checkpoint/'+model_chk_name+'.pth')
model.load_state_dict(checkpoint['model'])
train_losses = checkpoint['train_loss']
test_losses = checkpoint['test_loss']
train_accs = checkpoint['train_acc']
test_accs = checkpoint['test_acc']
lrs = checkpoint['lr']
best_acc = 0.0 if not test_accs else max(test_accs)
start_epoch = checkpoint['epoch']
print("\n Starting epoch : ",start_epoch)
print("\n Starting best_acc : ",best_acc)


 Starting epoch :  -1

 Starting best_acc :  0.0


In [14]:
for epoch in range(start_epoch +1 , start_epoch + 201):
    train(epoch)
    test(epoch)
    lrs.append(optimizer.param_groups[0]['lr'])
    scheduler.step(test_accs[-1])


Epoch: 0
Train | Loss: 0.0168 | Acc: 21.98
Test  | Loss: 0.0145 | Acc: 29.58

Epoch: 1
Train | Loss: 0.0136 | Acc: 34.72
Test  | Loss: 0.0125 | Acc: 41.68

Epoch: 2
Train | Loss: 0.012 | Acc: 43.49
Test  | Loss: 0.0127 | Acc: 41.98

Epoch: 3
Train | Loss: 0.0106 | Acc: 50.89
Test  | Loss: 0.0098 | Acc: 54.32

Epoch: 4
Train | Loss: 0.0093 | Acc: 57.76
Test  | Loss: 0.0116 | Acc: 51.94

Epoch: 5
Train | Loss: 0.0083 | Acc: 62.67
Test  | Loss: 0.0084 | Acc: 62.17

Epoch: 6
Train | Loss: 0.0075 | Acc: 66.47
Test  | Loss: 0.0075 | Acc: 66.8

Epoch: 7
Train | Loss: 0.0069 | Acc: 69.48
Test  | Loss: 0.0079 | Acc: 66.77

Epoch: 8
Train | Loss: 0.0064 | Acc: 71.91
Test  | Loss: 0.0086 | Acc: 64.46

Epoch: 9
Train | Loss: 0.0061 | Acc: 73.43
Test  | Loss: 0.0064 | Acc: 72.62

Epoch: 10
Train | Loss: 0.0057 | Acc: 75.29
Test  | Loss: 0.0084 | Acc: 67.74

Epoch: 11
Train | Loss: 0.0056 | Acc: 75.84
Test  | Loss: 0.0063 | Acc: 73.03

Epoch: 12
Train | Loss: 0.0054 | Acc: 76.65
Test  | Loss: 0.008

In [ ]:
!pip install matplotlib
import matplotlib.pyplot as plt
plt.figure(figsize = (10,3))
plt.subplot(1, 2, 1)
plt.plot(range(len(train_losses)), train_losses, label = 'Train loss')
plt.plot(range(len(test_losses)), test_losses, label = 'Val loss')
plt.plot(range(len(lrs)), lrs, label = 'learning rate')
plt.title('Loss curve')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.grid()
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(range(len(train_losses)), train_accs, label = 'Train acc')
plt.plot(range(len(test_losses)), test_accs, label = 'Val acc')
plt.plot(range(len(lrs)), lrs, label = 'learning rate')
plt.title('Accuracy curve')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.grid()
plt.legend()
plt.tight_layout()
plt.show()

The loss should have divided by the number of samples...

In [ ]:
best_acc